## Pipeline API

The Pipeline API is a powerful tool for cross-sectional analysis of asset data. It allows us to define a set of calculations on multiple data inputs and analyze a large amount of assets at a time. Some common uses for the Pipeline API include:  

* Selecting assets based on filtering rules  
* Ranking assets based on a scoring function  
* Calculating portfolio allocations  

To begin, let's import the Pipeline class and create a function that returns an empty pipeline. Putting our pipeline definition inside a function helps us keep things organized as our pipeline grows in complexity. This is particularly helpful when transferring data pipelines between Research and the IDE.

In [1]:
# Pipeline class
from quantopian.pipeline import Pipeline

def make_pipeline():
    # Create and return an empty Pipeline
    return Pipeline()

To add an output to our pipeline we need to include a reference to a dataset, and specify the computations we want to carry out on that data. For example, we will add a reference to the `close` column from the `USEquityPricing` dataset. Then, we can define our output to be the latest value from this column as follows:

In [2]:
# Import Pipeline class and USEquityPricing dataset
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing

def make_pipeline():
    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Return Pipeline containing latest closing price
    return Pipeline(
        columns={
            'close_price': close_price,
        }
    )

The Pipeline API also provides a number of built-in calculations, some of which are computed over trailing windows of data. For example, the following code imports Psychsignal's `stocktwits` dataset and defines an output as the 3 day moving average of its `bull_minus_bear` column:

In [3]:
# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.psychsignal import stocktwits

# Import built-in moving average calculation
from quantopian.pipeline.factors import SimpleMovingAverage


def make_pipeline():
    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Calculate 3 day average of bull_minus_bear scores
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )

    # Return Pipeline containing close_price
    # and sentiment_score
    return Pipeline(
        columns={
            'close_price': close_price,
            'sentiment_score': sentiment_score,
        }
    )

### Universe Selection

An important part of developing a strategy is defining the set of assets that we want to consider trading in our portfolio. We usually refer to this set of assets as our trading universe.  

A trading universe should be as large as possible, while also excluding assets that aren't appropriate for our portfolio. For example, we want to exclude stocks that are illiquid or difficult to trade. Quantopian's `QTradableStocksUS` universe offers this characteristic. We can set `QTradableStocksUS` as our trading universe using the screen parameter of our pipeline constructor:

In [4]:
# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.psychsignal import stocktwits

# Import built-in moving average calculation
from quantopian.pipeline.factors import SimpleMovingAverage

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Calculate 3 day average of bull_minus_bear scores
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )

    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
            'sentiment_score': sentiment_score,
        },
        screen=base_universe
    )

Now that our pipeline definition is complete, we can execute it over a specific period of time using `run_pipeline`. The output will be a pandas DataFrame indexed by date and asset, with columns corresponding to the outputs we added to our pipeline definition:

In [7]:
# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2020-01-01',
    end_date='2020-04-10'
)

# Display last 10 rows
pipeline_output.tail(100)

close_price  sentiment_score
2020-04-13 00:00:00+00:00 Equity(52084 [VNE])         8.960         0.000000
                          Equity(52100 [AVLR])       73.760        -0.876000
                          Equity(52115 [EIDX])       46.430         0.000000
                          Equity(52119 [TCDA])       26.720         0.151667
                          Equity(52121 [EPRT])       12.940        -0.150000
                          Equity(52127 [AVRO])       13.980         0.000000
                          Equity(52128 [IIIV])       21.420         0.000000
                          Equity(52143 [EVER])       35.480        -0.691667
                          Equity(52159 [BJ])         25.140         0.936667
                          Equity(52186 [AFIN])        6.930        -0.150000
                          Equity(52200 [RPAY])       14.960         0.440000
                          Equity(52209 [ALLK])       63.180         0.443333
                          Equity(52211 [TLRY])        6.750         0.045442
                          Equity(52213 [CNST])       34.180         0.000000
                          Equity(52230 [PRVB])        8.830         0.204167
                          Equity(52233 [BE])          6.145         0.347889
                          Equity(52241 [TENB])       22.220         2.530000
                          Equity(52242 [FOCS])       20.700         0.000000
                          Equity(52281 [VRT])         9.180         0.000000
                          Equity(52288 [CWK])        12.205         0.036667
                          Equity(52291 [SONO])        9.190        -1.333333
                          Equity(52361 [BPYU])       10.710              NaN
                          Equity(52404 [PRNB])       64.180         0.733333
                          Equity(52407 [FTDR])       36.225        -0.450000
                          Equity(52424 [ELAN])       24.580        -0.150000
                          Equity(52425 [FTCH])       10.002         1.345000
                          Equity(52427 [EB])          8.500         0.920000
                          Equity(52428 [YMAB])       29.130         0.000000
                          Equity(52446 [UPWK])        7.285         0.988000
                          Equity(52448 [ARCE])       43.320         0.000000
...                                                     ...              ...
                          Equity(53089 [PINS])       16.750         0.955663
                          Equity(53095 [ZM])        124.110        -0.207103
                          Equity(53116 [BYND])       72.630        -0.080837
                          Equity(53134 [PSN])        35.990         0.000000
                          Equity(53145 [NXTC])       40.300         0.000000
                          Equity(53148 [KTB])        20.270         0.583333
                          Equity(53158 [UBER])       27.090        -0.068408
                          Equity(53179 [FSLY])       20.840         0.015556
                          Equity(53181 [AVTR])       13.900        -0.146667
                          Equity(53196 [CTVA])       27.980        -0.333333
                          Equity(53238 [RVLV])       10.080         0.917222
                          Equity(53244 [AMCR])        9.160              NaN
                          Equity(53248 [CRWD])       58.240         0.402910
                          Equity(53251 [FVRR])       30.310         1.170417
                          Equity(53256 [CHWY])       42.680         0.216700
                          Equity(53260 [IAA])        32.085         0.000000
                          Equity(53267 [STOK])       21.740         0.000000
                          Equity(53271 [WORK])       24.740         0.671997
                          Equity(53273 [GO])         33.860        -0.723333
                          Equity(53293 [BSTZ])       18.350         1.416667
                          Equity(53303 [BBIO

In the next lesson we will formalize the strategy our algorithm will use to select assets to trade. Then, we will use a factor analysis tool to evaluate the predictive power of our strategy over historical data.